## Brain age regression using NIDL

In this notebook, we will show how to train a brain age model on OpenBHB (ROI measures) using :

1) standard L1 model
2) more advanced contrastive Rank-N-Constrast (RnC) model

During training, the different regression metrics (Pearson-r, $R^2$, RMSE, MAE) will be monitored on the validation set using Callback.

## Install and load the packages

First, we need to install some Python packages used by NIDL (if not already done):

In [ ]:
%pip install -e ../

In [1]:
from nidl.datasets.openbhb import OpenBHB
from nidl.models import DeepRegressor
from nidl.callbacks import RegressionMetricsCallback
from nidl.transform import Transform
from pytorch_lightning.loggers import CSVLogger 
from nilearn import plotting, datasets
import nibabel
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import seaborn
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

## Fit L1 model and monitor the validation metrics

In [ ]:
# Load the data
dataset_train = OpenBHB("/neurospin/signatures/bd261576/openBHB", 
                        target="age", modality="vbm_roi", split="train")
dataset_val = OpenBHB("/neurospin/signatures/bd261576/openBHB",
                      target="age", modality="vbm_roi", split="val")
train_dataloader = DataLoader(dataset_train, batch_size=128, num_workers=10, shuffle=True)
val_dataloader = DataLoader(dataset_val, batch_size=128, num_workers=10, shuffle=False)

In [6]:
# Fit L1
model = DeepRegressor(
    encoder="mlp",
    encoder_kwargs={"layers": [284, 128, 128, 1], "n_embedding": 1},
    loss="l1",
    learning_rate=1e-3,
    callbacks=[RegressionMetricsCallback()],
    logger=CSVLogger(".")
)
model.fit(train_dataloader)

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home_local/bd261576/miniconda3/envs/nidl/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/home_local/bd261576/miniconda3/envs/nidl/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CU

Epoch 0:  23%|██▎       | 6/26 [00:04<00:13,  1.43it/s, v_num=1, loss/train=24.10]

/home_local/bd261576/miniconda3/envs/nidl/lib/python3.10/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 0: 100%|██████████| 26/26 [00:11<00:00,  2.17it/s, v_num=1, loss/train=22.00]

/home_local/bd261576/miniconda3/envs/nidl/lib/python3.10/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


PicklingError: Can't pickle <function <lambda> at 0x70fb0ddcb1c0>: attribute lookup <lambda> on __main__ failed